In [9]:
import requests
from bs4 import BeautifulSoup

## 1. Web scraping 

In [10]:
def getPage(url):

    session = requests.Session()
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
    try:
        req = session.get(url, headers=headers)
    except requests.exceptions.RequestException:
        return None
    bs = BeautifulSoup(req.text, 'html.parser')
    return bs



In [13]:
import requests

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body


def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')


def scrapeElDiario(url):
    bs = getPage(url)
    title = bs.find('h1').text
    lines = bs.select('p.article-text')
    body = '\n'.join([line.text for line in lines])
    return Content(url, title, body)


url = 'https://www.eldiario.es/cultura/arte/dia-durero-ver-ballena-encontro-muerte_1_8571286.html'
content = scrapeElDiario(url)
print('Title: {}'.format(content.title))
print('URL: {}\n'.format(content.url))
print(content.body)

Title: 
                        El día que Durero fue a ver una ballena y encontró la muerte
        
URL: https://www.eldiario.es/cultura/arte/dia-durero-ver-ballena-encontro-muerte_1_8571286.html


        "Hace quinientos años, Durero llegó a este lugar en busca de una ballena. Aquel fue el punto de inflexión de su vida". Cinco siglos después, Philip Hoare (Southampton, Reino Unido, 63 años) y su pareja, Ellen, llegaban a Róterdam (Holanda) en coche persiguiendo el rastro de la leyenda, la última parte de la vida del artista alemán y, como él mismo dice, una gran aventura de arte, animales, decadencia, poesía y mar. La historia arranca donde lo hacen todas, en la falta de dinero. En 1519, el artista más famoso al norte de Italia estaba preocupado: ya no era joven y el cuerpo comenzaba a fallarle, temía perder la vista y la destreza. Y su mecenas, Maximiliano I, emperador del sacro Imperio Romano, había muerto. Se había quedado sin ingresos regulares y las garantías, de repente, habí

## 2. Web crawling

In [21]:
class Content:

    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

    def print(self):
        print('URL: {}'.format(self.url))
        print('TITLE: {}'.format(self.title))
        print('BODY:\n{}'.format(self.body))

class Website:

    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [22]:
class Crawler:

    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def safeGet(self, pageObj, selector):
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
        return ''

    def parse(self, site, url):
        bs = self.getPage(url)
        if bs is not None:
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            if title != '' and body != '':
                content = Content(url, title, body)
                content.print()

In [25]:
crawler = Crawler()

siteData = [
    ['New York Times', 'http://nytimes.com', 'h1', 'div.StoryBodyCompanionColumn div p']
]
websites = []
for row in siteData:
    websites.append(Website(row[0], row[1], row[2], row[3]))

crawler.parse(websites[0], 
    'https://www.nytimes.com/2018/01/28/business/energy-environment/oil-boom.html')

URL: https://www.nytimes.com/2018/01/28/business/energy-environment/oil-boom.html
TITLE: Oil Boom Gives the U.S. a New Edge in Energy and Diplomacy
BODY:
HOUSTON — A substantial rise in oil prices in recent months has led to a resurgence in American oil production, enabling the country to challenge the dominance of Saudi Arabia and dampen price pressures at the pump.
The success has come in the face of efforts by Saudi Arabia and its oil allies to undercut the shale drilling spree in the United States. Those strategies backfired and ultimately ended up benefiting the oil industry.
Overcoming three years of slumping prices proved the resiliency of the shale boom. Energy companies and their financial backers were able to weather market turmoil — and the maneuvers of the global oil cartel — by adjusting exploration and extraction techniques.
After a painful shakeout in the industry that included scores of bankruptcies and a significant loss of jobs, a steadier shale-drilling industry is a